# Cost Module

In [1]:
import openpyxl
import xlwings as xw
import math

In [ ]:
## Function calculating total cell cost for the pack over service life
def initialCellCost(deg_dict, cf, design_dict, cost_scale_factor):
    cost_dict={}
    cost_dict["tot_replacements"] = math.ceil(deg_dict['expected_life']/deg_dict['deg_life'])
    cost_dict["cost_cell"] = cf['Cost (GBP)']*cost_scale_factor
    cost_dict["cost_pack_cells"] = design_dict['tot_cells']*cost_dict["cost_cell"]
    
    return cost_dict

In [ ]:
## Refer chemistries to a certain cell model (for use in BatPaC)
batPacChem = {}
batPacChem['LFP'] = 'LFP-G'
batPacChem['NMC']= 'NMC622-G'
batPacChem['NCA']='NCA-G'
batPacChem['IMR']='LMO-G'
batPacChem['LTO']='LMO-LTO'
batPacChem['LCO'] = 'LFP-G'

In [7]:
 ## Inputting cell design into BatPaC
def input_BATPAC(c_path, design_dict, prod_rate, chem):
    wb = openpyxl.load_workbook(c_path, read_only = False, keep_vba = True)
    ws = wb['Dashboard']

    ws['D33'] = design_dict['max_power']*1000 # max discharge power required (kW)
    ws['D34'] = design_dict['tot_cells'] # number of cells total
    ws['D35'] = design_dict['tot_cells_parallel'] # number of cells per module in parallel
    ws['D39'] = math.ceil(prod_rate/design_dict['tot_cap']) # number of cells produced every year
    ws['D43'] = design_dict['tot_cap'] # capacity in kWh
    
    ws['E9'] = batPacChem[chem]
    
    # Activates excel VBA code when application is opened
    ws['D56'] = 0
    
    wb.save(c_path)
    return

In [ ]:
## Function to run VBA code and retrieve calculated battery pack cost
def packcost_BATPAC(c_path):
    wbxl = xw.Book(c_path)
    app = xw.apps.active
    
    add_mod_cost = wbxl.sheets['Summary of Results'].range('G67').value
    add_pack_cost = wbxl.sheets['Summary of Results'].range('G68').value
    
    try:
        add_cost = add_mod_cost + add_pack_cost
        wbxl.save()
        app.quit() # close application
    except:
        print('BATPAC ERROR')
        add_cost = 0
        app.quit()

    return add_cost

In [4]:
## Calculating Total Service Cost Values
def agg_costs(expected_life, deg_life, normal_cost, capacity_kWh):
    tot_replacements = math.ceil(expected_life/deg_life)
    tot_cost = tot_replacements*normal_cost*capacity_kWh/1000000 ##in £M
    return tot_cost, tot_replacements